In [ ]:
import json
import pandas as pd
import numpy as np
import itertools
import nltk
from nltk import tokenize
from nltk.tokenize import word_tokenize

with open('nz-annotated.json1', 'r') as json_file:
    json_list = list(json_file)
result = []
for json_str in json_list:
    result.append(json.loads(json_str))

In [ ]:
def itersplit(l, splitters):
    current = []
    for item in l:
        if item[0] in splitters:
            current.append(item)
            yield current
            current = []
        else:
            current.append(item)
    yield current

def sentsplit(l, splitters):
    return [subl for subl in itersplit(l, splitters) if subl]
    
df = pd.DataFrame(columns = ['DocID','sent_id','pos','token','tags'])
sentid=0
for doc in result:
    l = doc.get('labels')
    l.sort(key=lambda x:x[1])
    remdup = []
    for i in l:
        if i not in remdup:
            remdup.append(i)
    
    track = list() #handle doubles
    uniremdup = list()
    for elm in remdup:
        if elm[0] not in track:
            track.append(elm[0])
            uniremdup.append(elm)
    remdup = uniremdup #handle doubles
    
    tdoc = doc.get('text')
    todf = list()
    docID = tdoc[0:6]
    prevtoken=[0,0,'']
    for label in remdup:
        prevtoken=tdoc[prevtoken[1]:label[0]]
#       splits = prevtoken.split()
        splits = word_tokenize(prevtoken)
        if splits:
            othEnt = list(zip(*[splits,['O']*len(splits)]))
            todf.append(othEnt)
        token=tdoc[label[0]:label[1]]
        token = token.replace("\r\n"," ")
        #nEnt = [(token,label[2])]
#       nEntSplit = token.split()
        nEntSplit = word_tokenize(token)
        nEnt = list(zip(*[nEntSplit,[label[2]]*len(nEntSplit)]))
        todf.append(nEnt)
        prevtoken = label
        if label == l[-1]:
#           lasttokens = tdoc[l[-1][1]:].split()
            lasttokens = word_tokenize(tdoc[l[-1][1]:])
            lastEnt = list(zip(*[lasttokens,['O']*len(lasttokens)]))
            todf.append(lastEnt)
            
    reshaped = list(itertools.chain.from_iterable(todf))
    simplesa = list(map(lambda a:a[0], reshaped))
    postags = nltk.pos_tag(simplesa, tagset='universal')
    entwpos = [(i[0],i[1], j[1]) for i, j in zip(reshaped,postags)]

    text = doc.get('text')
    text=text.replace("\r\n"," ")
    text=' '.join(text.split())
    sentences=tokenize.sent_tokenize(text)
#   endsent = [string.split()[-1] for string in sentences]
    endsent = [word_tokenize(string) [-1] for string in sentences]
    sentences = sentsplit(entwpos, endsent)

    for idx,sentn in enumerate(sentences):
        ids = len(sentn)*[sentid]
        sentid = sentid+1
        #ids = len(sentn)*[str(idx)]
        dids = len(sentn)*[docID]
        unwrappedl = list(zip(*sentn))
        data = pd.DataFrame({'DocID': dids,'sent_id': ids, 'pos': unwrappedl[2], 'token': unwrappedl[0], 'tags': unwrappedl[1]})
        df = df.append(data,ignore_index=True)

In [ ]:
# sliding window to convert df to IOB
from itertools import islice
def getTag(result):
    if result[1] != result[0] and result[1] != result[2] and result[1] != 'O':
        return 'U-'+result[1]
    elif result[1] != result[0] and result[1] == result[2] and result[1] != 'O':
        return 'B-'+result[1]
    elif result[1] == result[0] and result[1] == result[2] and result[1] != 'O':
        return 'I-'+result[1]
    elif result[1] == result[0] and result != result[2] and result[1] != 'O':
        return 'L-'+result[1]
    else:
        return result[1]

def window(seq):
    it = iter(seq)
    lastn = len(seq)
    result = tuple(islice(it, 3))
    if len(result) == 3:
        if result[0] != result[1] and result[0] != 'O':
             tag = 'U-'+result[0]
        elif result[0] == result[1] and result[0] != 'O':
            tag = 'B-'+result[0]
        else:
            tag = result[1]
        yield tag
    yield getTag(result)
    
    for idx, elem in enumerate(it):
        result = result[1:] + (elem,) 
        yield getTag(result)
        
    if result[2] != result[1] and result[2] != 'O':
        tag = 'U-'+result[2]
    elif result[2] == result[1] and result[2] != 'O':
        tag = 'L-'+result[2]
    else:
        tag = result[2]
    yield tag

In [ ]:
df = df[df.tags != 'DocID_Internal']

In [ ]:
iobtags = list(window(df['tags']))
df['tags'] = iobtags

In [ ]:
import pickle
df.to_pickle("nz_data_df_new.p")

In [ ]:
classCount = df.groupby('tags',as_index=False).count()
classCount

,tags,DocID,sent_id,pos,token
0,B-CONTACT,29,29,29,29
1,B-DATE,14,14,14,14
2,B-ID,1,1,1,1
3,B-LOCATION,55,55,55,55
4,B-NAME,67,67,67,67
5,B-PROFESSION,1,1,1,1
6,I-CONTACT,118,118,118,118
7,I-DATE,10,10,10,10
8,I-LOCATION,22,22,22,22
9,I-NAME,12,12,12,12


In [ ]:
df.pos.unique()

array(['.', 'NOUN', 'ADP', 'NUM', 'PRON', 'VERB', 'DET', 'ADJ', 'ADV',
       'CONJ', 'PRT', 'X'], dtype=object)